In [26]:
#!pip install selenium
#!pip install webdriver_manager
from selenium import webdriver

import os
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

import codecs
from selenium.webdriver.common.keys import Keys
import re

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select

In [27]:
website = 'https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating'
driver = webdriver.Chrome()
driver.get(website)

In [28]:
next_page = driver.find_element(By.LINK_TEXT, 'Next »')
for x in driver.find_elements(By.XPATH,'//*[@id="main"]/div/div[4]/span[1]'):
    end = int(x.text.split(' ')[2].replace(',',''))
rang = []
film = []
jahr = []
fsk = []
dauer = []
genre = []
bewertung = []
regisseur = []
stars = []

while next_page:
    headers = driver.find_elements(By.TAG_NAME, 'h3')
    for head in headers:
        rang.append(head.text.split(' ')[0])
        jahr.append(head.text.split(' ')[-1])
        film.append(head.text.split(' ')[1:-1])
    rang = rang[:-1]
    jahr = jahr[:-1]
    film = film[:-1]

    genre_infos = driver.find_elements(By.XPATH, '//p[@class="text-muted "]')
    for genre_info in genre_infos:
        if len(genre_info.text.split('|')) == 3:
            fsk.append(genre_info.text.split('|')[0])
            dauer.append(genre_info.text.split('|')[1])
            genre.append(genre_info.text.split('|')[2])
        elif len(genre_info.text.split('|')) == 2:
            fsk.append('NaN')
            dauer.append(genre_info.text.split('|')[0])
            genre.append(genre_info.text.split('|')[1])

    ratings_bar =  driver.find_elements(By.CLASS_NAME, 'ratings-bar')
    for rating_bar in ratings_bar:
        bewertung.append(rating_bar.text.split(' ')[0])

    movie_info = driver.find_elements(By.XPATH, '//div[@class="lister-item-content"]')
    for info in movie_info:
        # Split the text into lines and select the second line
        lines = info.text.split('\n')
        staff = lines[-2].split(" | ")
        # Iterating through the list and replacing 'Director:' with an empty string
        for i in range(len(staff)):
            staff[i] = staff[i].replace('Director: ', '')
            staff[i] = staff[i].replace('Stars: ', '')
        # Append the duration to the list
        regisseur.append(staff[0])
        stars.append(staff[-1])

    if len(rang)== end:
        break    
    next_page = driver.find_element(By.LINK_TEXT, 'Next »')
    next_page.click()

driver.quit()

# lists to dataframe
import pandas as pd
df = pd.DataFrame()
df['rang'] = rang
df['film'] = film
df['jahr'] = jahr
df['fsk'] = fsk
df['dauer'] = dauer
df['genre'] = genre
df['bewertung'] = bewertung
df['regisseur'] = regisseur
df['stars'] = stars
df

,rang,film,jahr,fsk,dauer,genre,bewertung,regisseur,stars
0,1.,"[Die, Verurteilten]",(1994),12,142 min,Drama,"9,3",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi..."
1,2.,"[Der, Pate]",(1972),16,175 min,"Crime, Drama","9,2",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Diane Ke..."
2,3.,"[The, Dark, Knight]",(2008),16,152 min,"Action, Crime, Drama","9,0",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M..."
3,4.,"[Schindlers, Liste]",(1993),12,195 min,"Biography, Drama, History","9,0",Steven Spielberg,"Liam Neeson, Ralph Fiennes, Ben Kingsley, Caro..."
4,5.,"[Der, Herr, der, Ringe:, Die, Rückkehr, des, K...",(2003),12,201 min,"Action, Adventure, Drama","9,0",Peter Jackson,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or..."
...,...,...,...,...,...,...,...,...,...
995,996.,[Control],(2007),12,122 min,"Biography, Drama, Music","7,6",Anton Corbijn,"Sam Riley, Samantha Morton, Craig Parkinson, A..."
996,997.,"[Philomena:, Eine, Mutter, sucht, ihren, Sohn]",(2013),6,98 min,"Biography, Comedy, Drama","7,6",Stephen Frears,"Judi Dench, Steve Coogan, Sophie Kennedy Clark..."
997,998.,"[Shine, -, Der, Weg, ins, Licht]",(1996),12,105 min,"Biography, Drama, Music","7,6",Scott Hicks,"Geoffrey Rush, Armin Mueller-Stahl, Justin Bra..."
998,999.,"[Der, Unsichtbare]",(1933),16,71 min,"Horror, Sci-Fi","7,6",James Whale,"Claude Rains, Gloria Stuart, William Harrigan,..."


In [ ]:
df1 = df.copy()

In [ ]:
df1

In [ ]:
df1.rang = df1.rang.str.replace('.','')
#from film remove the brackets and commas regex
df1.film = df1['film'].str.join(' ')
df1.jahr = df1.jahr.str[1:-1]
df1.dauer = df1.dauer.str.replace('min','')
df1

In [ ]:
# jahr as date
df1['jahr'] = pd.to_datetime(df1['jahr'], format='%Y')

# EDA

In [ ]:
import seaborn as sns
#scatterplot jahr and bewertung
sns.scatterplot(data=df1, x="jahr", y="bewertung")

In [ ]:
driver.quit()